# Interaction and sampling demo v1

This notebook estimates a simple workplace location choice model to demonstrate some new discrete choice functionality in ChoiceModels [PR #37](https://github.com/UDST/choicemodels/pull/37) and UrbanSim Templates [PR #30](https://github.com/UDST/urbansim_templates/pull/30).

The headline feature is that we can now do sampling of alternatives using interaction weights that apply to particular combinations of choosers and alternatives.

In [33]:
import numpy as np
import pandas as pd

import warnings; warnings.simplefilter('ignore')

### Load home and workplace census tracts for CHTS survey participants

data download: https://www.nrel.gov/transportation/secure-transportation-data.html

In [2]:
import zipfile

In [3]:
z = zipfile.ZipFile('/Users/maurer/Dropbox/Data/CHTS/public/caltrans_full_survey.zip')

In [4]:
# Load households tables, which contains home locations

households = pd.read_csv(z.open('caltrans_full_survey/survey_households.csv'), low_memory=False)
len(households)

42426

In [5]:
# Clean and limit to Bay Area

households = households.loc[~households.home_tract_id.isnull() &
                            households.home_county_id.isin([1,13,41,55,75,81,85,95,97])]

households['home_tract_id'] = households.home_tract_id.astype(int)
len(households)

9715

In [6]:
# Load persons table, which contains employment locations

persons = pd.read_csv(z.open('caltrans_full_survey/survey_person.csv'), low_memory=False)
len(persons)

109113

In [7]:
# Clean and limit to Bay Area

persons = persons.loc[~persons.empl_tract_id.isnull() &
                      (persons.empl_tract_id // 1e9 == 6) &
                      persons.empl_county_id.isin([1,13,41,55,75,81,85,95,97]) & 
                      persons.sampno.isin(households.sampno)]

persons['empl_tract_id'] = persons.empl_tract_id.astype(int)
len(persons)

10217

In [8]:
# Merge into a table with one observation per person

obs = pd.merge(persons[['sampno', 'empl_tract_id']],
               households[['sampno', 'home_tract_id']],
               how='left', on='sampno')  # 'sampno' is the household identifier
obs.index.name = 'obs_id'
obs = obs.drop('sampno', axis='columns')
len(obs)

10217

In [9]:
obs.head(3)

,empl_tract_id,home_tract_id
obs_id,,
0,6081608900,6081608800
1,6081984300,6081608800
2,6081613400,6081608800


In [10]:
# Calculate employment density for use as an explantory variable

empl_density = persons.groupby('empl_tract_id').perwgt.sum().rename('empl_density').to_frame()

In [11]:
# Build a table of alternative employment locations

alts = pd.DataFrame({'empl_tract_id': obs.empl_tract_id.unique()}).set_index('empl_tract_id')
alts = alts.join(empl_density, how='left')

len(alts)

1241

In [12]:
alts.head(3)

,empl_density
empl_tract_id,
6081608900,6.699330
6081984300,33.612902
6081613400,2.356557


### Load census tract outlines to get centroids and Euclidean distances

data download: https://www.census.gov/geo/maps-data/data/cbf/cbf_tracts.html

In [13]:
import geopandas as gpd

In [14]:
# Load cartographic tract outlines

path = '/Users/maurer/Dropbox/Data/Census shapefiles/gz_2010_06_140_00_500k/gz_2010_06_140_00_500k.shp'
tgeom = gpd.read_file(path)
len(tgeom)

8048

In [15]:
# Calculate lat-lon and projected centroids

tgeom['centroid_x'] = tgeom.centroid.x
tgeom['centroid_y'] = tgeom.centroid.y

tgeom = tgeom.to_crs({'init': 'epsg:26910'})  # UTM 10N, units are meters
tgeom['x_proj'] = tgeom.centroid.x
tgeom['y_proj'] = tgeom.centroid.y

In [16]:
# Convert the tract id's

tgeom['tract_id'] = tgeom.GEO_ID.apply(lambda x: int(x[-10:]))

tgeom = tgeom[['tract_id', 'centroid_x', 'centroid_y', 'x_proj', 'y_proj']].set_index('tract_id')
tgeom.head(3)

,centroid_x,centroid_y,x_proj,y_proj
tract_id,,,,
6001425103,-122.289857,37.842579,562482.092186,4.188587e+06
6001425104,-122.283413,37.832637,563057.536773,4.187488e+06
6001426100,-122.225388,37.821620,568173.854727,4.186306e+06


In [17]:
# Limit to Bay Area

tgeom = tgeom.loc[tgeom.index.isin(obs.home_tract_id) |
                  tgeom.index.isin(obs.empl_tract_id)]
len(tgeom)

1549

In [18]:
# Check that all the CHTS tracts are covered

print(sum(~obs.home_tract_id.isin(tgeom.index.values)))
print(sum(~obs.empl_tract_id.isin(tgeom.index.values)))

0
0


### Calculate a matrix of Euclidean distances between tracts

In [19]:
from choicemodels.tools.distancematrix import distance_matrix

/Users/maurer/anaconda3/envs/dissertation-env/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [20]:
%%time
data = tgeom[['x_proj', 'y_proj']]
dm = distance_matrix(data, x='x_proj', y='y_proj')

CPU times: user 104 ms, sys: 56.5 ms, total: 161 ms
Wall time: 152 ms


In [21]:
type(dm)

pandas.core.series.Series

In [22]:
dm.index.set_names(('home_tract_id', 'empl_tract_id'), inplace=True)
dm = dm.rename('distance_km').to_frame()
dm = dm / 1000

print(len(dm))
dm.head(3)

2399401


distance_km
home_tract_id empl_tract_id             
6001425103    6001425103        0.000000
              6001425104        1.240268
              6001426100        6.131537

### Example 1: Uniform sampling of alternatives, with distance as an interaction term

In [23]:
from choicemodels import MultinomialLogit
from choicemodels.tools import MergedChoiceTable

In [24]:
%%time
mct = MergedChoiceTable(obs, alts, chosen_alternatives='empl_tract_id',
                        sample_size=10, interaction_terms=dm)

CPU times: user 490 ms, sys: 165 ms, total: 654 ms
Wall time: 595 ms


In [25]:
len(mct.to_frame())

102170

In [26]:
mct.to_frame().head(3)

home_tract_id  empl_density  chosen  distance_km
obs_id empl_tract_id                                                  
10216  6081605000        6081608100     19.079771       1    11.504308
       6085509403        6081608100      2.703642       0    23.242265
       6085507205        6081608100      1.455352       0    41.858110

In [27]:
# Estimate a model using ChoiceModels directly

model = MultinomialLogit(data=mct.to_frame(),
                         observation_id_col=mct.observation_id_col,
                         choice_col=mct.choice_col,
                         model_expression='empl_density + distance_km - 1')

model.fit()

/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:219: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._data[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:222: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)


                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:         10,217
Model:         Multinomial Logit   Df Residuals:             10,215
Method:       Maximum Likelihood   Df Model:                      2
Date:                 2018-08-27   Pseudo R-squ.:             0.566
Time:                      17:33   Pseudo R-bar-squ.:         0.566
AIC:                  20,441.334   Log-Likelihood:      -10,218.667
BIC:                  20,455.798   LL-Null:             -23,525.512
                  coef   std err          z     P>|z|   Conf. Int.
------------------------------------------------------------------
empl_density    0.0370     0.000    110.344     0.000             
distance_km    -0.0898     0.001   -152.994     0.000             

### Example 2: Using distance to generate sampling weights

In [47]:
# Make distant alternatives less likely to appear in the choice set

w = (dm**-0.3).clip(upper=1.0).distance_km.rename('w').to_frame()

In [48]:
w.head(3)

w
home_tract_id empl_tract_id          
6001425103    6001425103     1.000000
              6001425104     0.937444
              6001426100     0.580402

In [49]:
# For now, weights have to be specified with respect to the observation id
# rather than based on derivative characteristics like the home census tract

ids = obs.reset_index()[['obs_id','home_tract_id']].assign(temp=1)\
         .merge(alts.reset_index()[['empl_tract_id']].assign(temp=1))\
         .drop('temp', axis='columns')  # cartesian product of the obs and alt ids

weights = ids.merge(w, how='left', on=['home_tract_id', 'empl_tract_id'])\
             .drop('home_tract_id', axis='columns').set_index(['obs_id', 'empl_tract_id'])

weights.head(3)

w
obs_id empl_tract_id          
0      6081608900     0.917810
       6081984300     0.460742
       6081613400     0.482608

In [50]:
print(len(weights))
print(len(obs) * len(alts))

12679297
12679297


In [51]:
%%time
mct = MergedChoiceTable(obs, alts, chosen_alternatives='empl_tract_id',
                        sample_size=10, weights=weights.w, interaction_terms=dm)

CPU times: user 14 s, sys: 289 ms, total: 14.3 s
Wall time: 14.3 s


In [52]:
len(mct.to_frame())

102170

In [53]:
# Estimate a model using ChoiceModels directly

model = MultinomialLogit(data=mct.to_frame(),
                         observation_id_col=mct.observation_id_col,
                         choice_col=mct.choice_col,
                         model_expression='empl_density + distance_km - 1')

model.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:         10,217
Model:         Multinomial Logit   Df Residuals:             10,215
Method:       Maximum Likelihood   Df Model:                      2
Date:                 2018-08-27   Pseudo R-squ.:             0.465
Time:                      17:51   Pseudo R-bar-squ.:         0.465
AIC:                  25,160.626   Log-Likelihood:      -12,578.313
BIC:                  25,175.090   LL-Null:             -23,525.512
                  coef   std err          z     P>|z|   Conf. Int.
------------------------------------------------------------------
empl_density    0.0348     0.000    115.660     0.000             
distance_km    -0.0745     0.001   -131.829     0.000             

This improves our ability to estimate the impact of non-distance factors (z-statistic of employment density is higher), while reducing the precision of the distance term.

### Example 3: Estimating the model in UrbanSim Templates

There is also now a backdoor to pass a MergedChoiceTable directly to the fit() method of the LargeMultinomialLogitStep template. Direct support for weights and interaction terms is in progress.

In [56]:
from urbansim_templates.models import LargeMultinomialLogitStep

In [57]:
m = LargeMultinomialLogitStep()

In [58]:
m.model_expression = 'empl_density + distance_km - 1'

In [59]:
m.fit(mct)

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:         10,217
Model:         Multinomial Logit   Df Residuals:             10,215
Method:       Maximum Likelihood   Df Model:                      2
Date:                 2018-08-27   Pseudo R-squ.:             0.465
Time:                      18:06   Pseudo R-bar-squ.:         0.465
AIC:                  25,160.626   Log-Likelihood:      -12,578.313
BIC:                  25,175.090   LL-Null:             -23,525.512
                  coef   std err          z     P>|z|   Conf. Int.
------------------------------------------------------------------
empl_density    0.0348     0.000    115.660     0.000             
distance_km    -0.0745     0.001   -131.829     0.000             
